In [12]:
#install dependencies 
#pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu113
#pip install torch-sparse -f https://data.pyg.org/whl/torch-1.12.1+cu113.html
#pip install torch-scatter -f https://data.pyg.org/whl/torch-1.12.1+cu113.html
#pip install torch-geometric
#pip install open3d
#pip install pandas

In [33]:
from pathlib import Path
import glob
import os
import numpy as np
import pandas as pd

import torch
import open3d as o3d

print(torch.__version__)

1.12.1+cu113


In [2]:
path = "C:/Users/aorhu/Masaüstü/ADLM/repo/BodyMeshGNN/1119738/body_surface_mesh.pt"

In [3]:
data_sample = torch.load(path)

In [4]:
data_sample.x

tensor([[0.0179, 0.3036, 0.7190],
        [0.0223, 0.3036, 0.7163],
        [0.0223, 0.2976, 0.7190],
        ...,
        [0.9911, 0.2738, 0.7245],
        [0.9911, 0.2798, 0.8044],
        [0.9911, 0.2857, 0.8044]])

In [142]:
data_sample.eid

'2119543'

In [6]:
# with open3d: point cloud
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(np.asarray(data_sample.x))

o3d.visualization.draw_geometries([pcd])

In [67]:
# with open3d: triangle mesh (need face info)
# mesh = o3d.geometry.TriangleMesh()
# mesh.vertices = o3d.utility.Vector3dVector(np.asarray(data_sample.x))
# mesh.triangles = o3d.utility.Vector3dVector(np.asarray(data_sample.edge))

AttributeError: 'GlobalStorage' object has no attribute 'edge'

In [9]:
# with open3d: graph
lines = o3d.utility.Vector2iVector(np.asarray(data_sample.edge_index.T))
line_set = o3d.geometry.LineSet(
        points=o3d.utility.Vector3dVector(np.asarray(data_sample.x)),
        lines=lines)

line_set.colors = o3d.utility.Vector3dVector([[1, 0, 0] for i in range(len(lines))])

o3d.visualization.draw_geometries([line_set])

In [15]:
# reading ID mapping
id_map_path = "C:/Users/aorhu/Masaüstü/ADLM/Bridge_eids_60520_87802.csv"

In [16]:
id_map = pd.read_csv(id_map_path)

In [139]:
id_map.dtypes

eid_60520    int64
eid_87802    int64
dtype: object

In [134]:
id_map["eid_87802"][id_map.index[id_map['eid_60520'] == 1000160]]

0    4160881
Name: eid_87802, dtype: int64

In [ ]:
#reading file directories for patients into dictionary

In [86]:
DATA_ROOT = Path("D:/ADLM Data/body_meshes/")

map_dict = {}

In [144]:
for file in os.listdir(str(DATA_ROOT)):
    _path = os.path.join(str(DATA_ROOT), file).replace('\\', '/')
    map_dict[file] = dict({'path': str(_path) +'/body_surface_mesh.pt', 'eid': "___" })

In [145]:
map_dict

{'1000071': {'path': 'D:/ADLM Data/body_meshes/1000071/body_surface_mesh.pt',
  'eid': '___'},
 '1000109': {'path': 'D:/ADLM Data/body_meshes/1000109/body_surface_mesh.pt',
  'eid': '___'},
 '1000125': {'path': 'D:/ADLM Data/body_meshes/1000125/body_surface_mesh.pt',
  'eid': '___'},
 '1000180': {'path': 'D:/ADLM Data/body_meshes/1000180/body_surface_mesh.pt',
  'eid': '___'},
 '1000191': {'path': 'D:/ADLM Data/body_meshes/1000191/body_surface_mesh.pt',
  'eid': '___'},
 '1000352': {'path': 'D:/ADLM Data/body_meshes/1000352/body_surface_mesh.pt',
  'eid': '___'},
 '1000456': {'path': 'D:/ADLM Data/body_meshes/1000456/body_surface_mesh.pt',
  'eid': '___'},
 '1000721': {'path': 'D:/ADLM Data/body_meshes/1000721/body_surface_mesh.pt',
  'eid': '___'},
 '1000770': {'path': 'D:/ADLM Data/body_meshes/1000770/body_surface_mesh.pt',
  'eid': '___'},
 '1000812': {'path': 'D:/ADLM Data/body_meshes/1000812/body_surface_mesh.pt',
  'eid': '___'},
 '1000900': {'path': 'D:/ADLM Data/body_meshes/100

In [85]:
len(map_dict)

9292